# H.E.S.S. with Gammapy

[H.E.S.S.](https://www.mpi-hd.mpg.de/hfm/HESS/) is an array of gamma-ray telescopes located in Namibia. Gammapy is regularly used and fully supports H.E.S.S. high level data analysis, after export to the current [open data level 3 format](https://gamma-astro-data-formats.readthedocs.io/).

The H.E.S.S. data is private, and H.E.S.S. analysis is mostly documented and discussed at https://hess-confluence.desy.de/ and in H.E.S.S.-internal communication channels. However, in 2018, a small sub-set of archival H.E.S.S. data was publicly released, called the [H.E.S.S. DL3 DR1](https://www.mpi-hd.mpg.de/hfm/HESS/pages/dl3-dr1/), the data level 3, data release number 1. This dataset is 50 MB in size and is used in many Gammapy analysis tutorials, and can be downloaded via [gammapy download](https://docs.gammapy.org/dev/scripts/index.html?highlight=download).

This notebook is a quick introduction to H.E.S.S. data and instrument responses and contains some specifics that are important for H.E.S.S. users:

- IRF formats and shapes
- How to handle safe energy and max offset
- EVENTS and GTI formats (e.g. how HESS 1, 2, configs, ... are handled)
- Link to HESS Confluence where data and help is available
- Analysis of data reduced with H.E.S.S. internal s/w



## DL3 DR1

This is how to access data and IRFs from the H.E.S.S. data level 3, data release 1.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

In [ ]:
from gammapy.data import DataStore
from gammapy.maps import MapAxis, Map
from gammapy.datasets import MapDatasetOnOff
from gammapy.makers.utils import make_theta_squared_table
from gammapy.visualization import plot_theta_squared_table

In [ ]:
data_store = DataStore.from_dir("$GAMMAPY_DATA/hess-dl3-dr1")

In [ ]:
data_store.info()

In [ ]:
data_store.obs_table[:2][["OBS_ID", "DATE-OBS", "RA_PNT", "DEC_PNT", "OBJECT"]]

In [ ]:
obs = data_store.obs(23523)

In [ ]:
obs.events.select_offset([0, 2.5] * u.deg).peek()

In [ ]:
obs.aeff.peek()

In [ ]:
obs.edisp.peek()

In [ ]:
obs.psf.peek()

In [ ]:
obs.bkg.to_2d().plot()

## Theta squared event distribution
As a quick look plot it can be helpful to plot the quadratic offset (theta squared) distribution of the events. 

In [ ]:
position = SkyCoord(ra=83.63, dec=22.01, unit="deg", frame="icrs")
theta2_axis = MapAxis.from_bounds(0, 0.2, nbin=20, interp="lin", unit="deg2")

observations = data_store.get_observations([23523, 23526])
theta2_table = make_theta_squared_table(
    observations=observations,
    position=position,
    theta_squared_axis=theta2_axis,
)

In [ ]:
plt.figure(figsize=(10, 5))
plot_theta_squared_table(theta2_table)

## Using 2D images created within internal HESS s/w

It is possible to use maps created outside gammapy, eg, using internal H.E.S.S. s/w `hap`, for modelling and fitting within gammapy. For this, it is necessary to attach an appropriate energy axis with a single bin on the maps, and also ensure that the maps have proper units.

In this example, we use some `hap` produced files shipped with `Gammapy-data`. Details of these files can be found in [this internal hess page](https://hess-confluence.desy.de/confluence/display/HESS/HGPS+Paper+-+Survey+Maps#HGPSPaperSurveyMaps-HAPLatestSur[…]sLatestHAPmapsandwhattheycontain). 

Note: **we strongly recommend that you re-process the files using gammapy.makers** to avoid unnecssary errors. 



In [ ]:
# Read the relevant hdus

counts = Map.read(
    "$GAMMAPY_DATA/tests/unbundled/hess/survey/hess_survey_snippet.fits.gz",
    hdu="On",
)
counts_off = Map.read(
    "$GAMMAPY_DATA/tests/unbundled/hess/survey/hess_survey_snippet.fits.gz",
    hdu="Off",
)
onExposure = Map.read(
    "$GAMMAPY_DATA/tests/unbundled/hess/survey/hess_survey_snippet.fits.gz",
    hdu="OnExposure",
)
offExposure = Map.read(
    "$GAMMAPY_DATA/tests/unbundled/hess/survey/hess_survey_snippet.fits.gz",
    hdu="OffExposure",
)
mask = Map.read(
    "$GAMMAPY_DATA/tests/unbundled/hess/survey/hess_survey_snippet.fits.gz",
    hdu="Exclusion",
)
expgamma = Map.read(
    "$GAMMAPY_DATA/tests/unbundled/hess/survey/hess_survey_snippet.fits.gz",
    hdu="ExpGammaMap",
)

Now, attach a relevant energy axis - a reconstructed axis on irfs, and true axis on the observed quantitites. 
Ideally, the energy range should be decided by your analysis cuts, we take an approximate value here.
We also have to attach an unit on the exposure - ensure to take it correctly.

In [ ]:
energy_min = 300 * u.GeV
energy_max = 10 * u.TeV
energy_axis = MapAxis.from_energy_edges(
    [energy_min, energy_max], name="energy"
)
energy_axis_true = MapAxis.from_energy_edges(
    [energy_min, energy_max], name="energy_true"
)

In [ ]:
counts = counts.to_cube([energy_axis])
counts_off = counts_off.to_cube([energy_axis])
acceptance = onExposure.to_cube([energy_axis])
acceptance_off = offExposure.to_cube([energy_axis])
mask = mask.to_cube([energy_axis])

exposure = expgamma.to_cube(
    [energy_axis_true]
)  # IRFs should have true energy axis
exposure.unit = u.cm * u.cm * u.s

In [ ]:
# In this example, there is no psf present. If present, make a cube as
# psf_map = psf.psf_kernel_map.to_cube([energy_axis_true])
# psf = PSFKernel(psf_map)

In [ ]:
# Create a dataset
dataset_onoff = MapDatasetOnOff(
    counts=counts,
    counts_off=counts_off,
    exposure=exposure,
    acceptance=acceptance,
    acceptance_off=acceptance_off,
    mask_fit=mask,
)

In [ ]:
dataset_onoff.exposure.plot(add_cbar=True)

You can perform a few sanity checks to ensure that the quantities computed by `hap` and `gammapy` are exactly the same.

In [ ]:
# the alpha map
alpha_hap = Map.read(
    "$GAMMAPY_DATA/tests/unbundled/hess/survey/hess_survey_snippet.fits.gz",
    hdu="Alpha",
)
alpha_gammapy = dataset_onoff.alpha
print(np.all(alpha_hap.data == alpha_gammapy.data))

In [ ]:
plt.figure(figsize=(12, 5))

ax1 = plt.subplot(121, projection=alpha_hap.geom.wcs)
ax2 = plt.subplot(122, projection=alpha_gammapy.geom.wcs)

alpha_hap.plot(add_cbar=True, ax=ax1)
alpha_gammapy.plot(add_cbar=True, ax=ax2)
ax1.set_title("alpha_hap")
ax2.set_title("alpha_gammapy");

You can now proceed with modelling and fitting of 2D images as explain in [this notebook](../../analysis/2D/modeling_2D..ipynb)

## Exercises

- Find the `OBS_ID` for the runs of the Crab nebula
- Compute the expected number of background events in the whole FOV for `OBS_ID=23523` in the 1 TeV to 3 TeV energy band, from the background IRF.

## Next steps

Now you know how to access and work with H.E.S.S. data. All other tutorials and documentation apply to H.E.S.S. and CTA or any other IACT that provides DL3 data and IRFs in the standard format.